In [1]:
import numpy as np
import pennylane as qml
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner

In [2]:
electrons_list = [2, 2, 4, 6, 8, 10, 12]
orbitals_list = [4, 6, 8, 12, 16, 20, 24]

# Here we will use the hydrogen molecule as an example:
electrons = 2
orbitals = 4
singles, doubles = qml.qchem.excitations(electrons, orbitals)

The chosen pair of (#electron, #orbital) represents valid active space of simulation of H$_2$, LiH, H$_2$O, C$_6$H$_6$, .., Cr$_2$. The orbital number is equivalent to the qubit number.

In [3]:
UCCSD_Paulis = []

# first we compute all the Paulis from fermionic single excitation
for i in singles:
    Paulis = jordan_wigner(FermionOperator(str(i[1])+'^ ' + str(i[0])))
    
    # The Paulis.terms save the Jordan-Wigner transformed single excitations in dictionary format:
    # key (tuple of tuples): A dictionary storing the coefficients of the terms in the operator. 
    # The keys are the terms. A term is a product of individual factors; each factor is represented
    # by a tuple of the form (index, action), and these tuples are collected into a larger tuple
    # which represents the term as the product of its factors.
    UCCSD_Paulis.append(Paulis.terms)
    
for i in doubles:
    Paulis = jordan_wigner(FermionOperator(str(i[2])+'^ '+str(i[3])+'^ '+str(i[1])+' '+str(i[0])))
    UCCSD_Paulis.append(Paulis.terms)

In [4]:
# converting to normal entangler format

entanglers = [] # list of lists

# rewrite the dictionary key into entangler form:
for Paulis_dict in UCCSD_Paulis:
    entangler_excitation = []
    
    for Paulis in list(Paulis_dict.keys()):    
        entangler = 'IIII'
        entangler_list = list(entangler)
        for Pauli_tuple in Paulis:
            entangler_list[Pauli_tuple[0]] = Pauli_tuple[1]
        entangler = ''.join(entangler_list)
        entangler_excitation.append(entangler)
    
    entanglers.append(entangler_excitation)
    
print(entanglers)

[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]


In [5]:
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

def construct_ucc_circuit(num_qubits: int, UCCSD_Paulis: list):
    '''This function defines the UCC ansatz circuit for VQE.
    
    Args:
        num_qubits: number of qubits in the circuit, equal to number of spin orbitals under jordan wigner mapping
        UCCSD_Paulis: list storing dictionary of Pauli words for constructing the UCC circuit
    Returns:
        ucc_circuit
    '''
    num_params = len(UCCSD_Paulis)
    p = ParameterVector('p', num_params)
    
    ucc_circuit = QuantumCircuit(num_qubits)
    
    for i in range(num_params):
        Paulis_dict = UCCSD_Paulis[i]
        circuit_block = QuantumCircuit(num_qubits)
        
        for Paulis in list(Paulis_dict.keys()):
            circuit = QuantumCircuit(num_qubits)
            
            # rewrite the dictionary key into entangler form:
            entangler = 'IIII'
            entangler_list = list(entangler)
            for Pauli_tuple in Paulis:
                entangler_list[Pauli_tuple[0]] = Pauli_tuple[1]
            entangler = ''.join(entangler_list)
            
            key = entangler
            coupler_map = []
            
            # We first construct coupler_map according to the key.
            for j in range(num_qubits):
                if key[num_qubits-1-j] != 'I':
                    coupler_map.append(j)

            # Then we construct the circuit.
            if len(coupler_map) == 1:
                # there is no CNOT gate.
                c = coupler_map[0]
                if key[num_qubits-1-c] == 'X':
                    circuit.h(c)
                    circuit.rz(p[i], c)
                    circuit.h(c)
                elif key[num_qubits-1-c] == 'Y':
                    circuit.rx(-np.pi/2, c)
                    circuit.rz(p[i], c)
                    circuit.rx(np.pi/2, c)
                circuit_block.compose(circuit, inplace=True)
                
            else:
                # Here we would need CNOT gate.
                for j in coupler_map:
                    if key[num_qubits-1-j] == 'X':
                        circuit.h(j)
                    elif key[num_qubits-1-j] == 'Y':
                        circuit.rx(-np.pi/2, j)

                for j in range(len(coupler_map) - 1):
                    circuit.cx(coupler_map[j], coupler_map[j+1])

                param_gate = QuantumCircuit(num_qubits)
                param_gate.rz(p[i], coupler_map[-1])
                
                circuit_block.compose(circuit, inplace=True)
                circuit_block.compose(param_gate, inplace=True)
                circuit_block.compose(circuit.inverse(), inplace=True)
                
        ucc_circuit.compose(circuit_block, inplace=True)
    
    return ucc_circuit

In [6]:
hf_circuit = QuantumCircuit(orbitals)

for i in range(electrons):
    hf_circuit.x(i)

hf_circuit.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────

In [7]:
parameterized_circuit = hf_circuit.compose(construct_ucc_circuit(orbitals, UCCSD_Paulis))
parameterized_circuit.draw()

┌───┐    ┌───┐                                                     »
q_0: ───┤ X ├────┤ H ├─────────────────────────────────────────────────────»
        ├───┤    ├───┤                                      ┌───┐┌───┐     »
q_1: ───┤ X ├────┤ H ├──■─────────────────────────────■─────┤ H ├┤ H ├──■──»
        └───┘    └───┘┌─┴─┐                         ┌─┴─┐   └───┘└───┘┌─┴─┐»
q_2: ─────────────────┤ X ├──■────────────────■─────┤ X ├─────────────┤ X ├»
     ┌──────────┐     └───┘┌─┴─┐┌──────────┐┌─┴─┐┌──┴───┴──┐┌───┐     └───┘»
q_3: ┤ Rx(-π/2) ├──────────┤ X ├┤ Rz(p[0]) ├┤ X ├┤ Rx(π/2) ├┤ H ├──────────»
     └──────────┘          └───┘└──────────┘└───┘└─────────┘└───┘          »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                                   ┌───┐    ┌──────────┐                      »
«q_1: ────────────────────────■─────┤ H ├────┤ Rx(-π/2) ├──■───────────────────»
«                           ┌─┴─┐   └───┘    └──────────┘┌─┴─┐                 »
«q_2: ──■────────────────■──┤ X ├────────────────────────┤ X ├──■──────────────»
«     ┌─┴─┐┌──────────┐┌─┴─┐├───┤┌──────────┐            └───┘┌─┴─┐┌──────────┐»
«q_3: ┤ X ├┤ Rz(p[0]) ├┤ X ├┤ H ├┤ Rx(-π/2) ├─────────────────┤ X ├┤ Rz(p[0]) ├»
«     └───┘└──────────┘└───┘└───┘└──────────┘                 └───┘└──────────┘»
«                                                                            »
«q_0: ───────────────────────────────────────────────────────────────────────»
«                     ┌─────────┐┌──────────┐                                »
«q_1: ──────────■─────┤ Rx(π/2) ├┤ Rx(-π/2) ├──■──────────────────────────■──»
«             ┌─┴─┐   └─────────┘└──────────┘┌─┴─┐                      ┌─┴─┐»
«q_2: ──■─────┤ X ├──────────────────────────┤ X ├──■────────────────■──┤ X ├»
«     ┌─┴─┐┌──┴───┴──┐   ┌───┐               └───┘┌─┴─┐┌──────────┐┌─┴─┐├───┤»
«q_3: ┤ X ├┤ Rx(π/2) ├───┤ H ├────────────────────┤ X ├┤ Rz(p[0]) ├┤ X ├┤ H ├»
«     └───┘└─────────┘   └───┘                    └───┘└──────────┘└───┘└───┘»
«                                                       ┌───┐┌───┐          »
«q_0: ──────────────■─────────────────────────────■─────┤ H ├┤ H ├──■───────»
«     ┌─────────┐ ┌─┴─┐                         ┌─┴─┐   └───┘└───┘┌─┴─┐     »
«q_1: ┤ Rx(π/2) ├─┤ X ├──■────────────────■─────┤ X ├─────────────┤ X ├──■──»
«     ├─────────┴┐└───┘┌─┴─┐┌──────────┐┌─┴─┐┌──┴───┴──┐┌───┐     └───┘┌─┴─┐»
«q_2: ┤ Rx(-π/2) ├─────┤ X ├┤ Rz(p[1]) ├┤ X ├┤ Rx(π/2) ├┤ H ├──────────┤ X ├»
«     ├──────────┤     └───┘└──────────┘└───┘└─────────┘└───┘          └───┘»
«q_3: ┤ Rx(-π/2) ├──────────────────────────────────────────────────────────»
«     └──────────┘                                                          »
«                              ┌───┐    ┌──────────┐                           »
«q_0: ───────────────────■─────┤ H ├────┤ Rx(-π/2) ├──■────────────────────────»
«                      ┌─┴─┐   └───┘    └──────────┘┌─┴─┐                      »
«q_1: ──────────────■──┤ X ├────────────────────────┤ X ├──■────────────────■──»
«     ┌──────────┐┌─┴─┐├───┤┌──────────┐            └───┘┌─┴─┐┌──────────┐┌─┴─┐»
«q_2: ┤ Rz(p[1]) ├┤ X ├┤ H ├┤ Rx(-π/2) ├─────────────────┤ X ├┤ Rz(p[1]) ├┤ X ├»
«     └──────────┘└───┘└───┘└──────────┘                 └───┘└──────────┘└───┘»
«q_3: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«                ┌─────────┐┌──────────┐                                »
«q_0: ─────■─────┤ Rx(π/2) ├┤ Rx(-π/2) ├──■──────────────────────────■──»
«        ┌─┴─┐   └─────────┘└──────────┘┌─┴─┐                      ┌─┴─┐»
«q_1: ───┤ X ├──────────────────────────┤ X ├──■────────────────■──┤ X ├»
«     ┌──┴───┴──┐   ┌───┐               └───┘┌─┴─┐┌──────────┐┌─┴─┐├───┤»
«q_2: ┤ Rx(π/2) ├───┤ H ├────────────────────┤ X ├┤ Rz(p[1]) ├┤ X ├┤ H ├»
